## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from g_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/updated_cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Chernyshevskiy,RU,63.0128,112.4714,37.80,97,100,6.22,overcast clouds
1,1,Tiksi,RU,71.6872,128.8694,5.16,100,66,3.22,broken clouds
2,2,Vaini,TO,-21.2000,-175.2000,60.96,94,20,5.75,few clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,93,0,6.91,clear sky
4,4,Svetlogorsk,BY,52.6333,29.7333,58.12,62,95,13.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)& \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
12,12,Georgetown,MY,5.4112,100.3354,82.33,94,20,0.00,moderate rain
14,14,Rikitea,PF,-23.1203,-134.9692,76.30,71,100,19.71,overcast clouds
15,15,Atuona,PF,-9.8000,-139.0333,78.53,75,18,18.68,few clouds
17,17,Sola,VU,-13.8833,167.5500,80.53,72,86,20.00,overcast clouds
18,18,Maceio,BR,-9.6658,-35.7353,83.64,69,75,9.22,broken clouds
19,19,Catuday,PH,16.2923,119.8062,82.74,72,100,3.18,overcast clouds
20,20,Mabaruma,GY,8.2000,-59.7833,75.22,98,100,2.82,overcast clouds
21,21,Madimba,TZ,-10.4000,40.3333,81.18,67,54,14.38,broken clouds
22,22,Acapulco,MX,16.8634,-99.8901,82.22,57,75,2.30,broken clouds
23,23,Karratha,AU,-20.7377,116.8463,77.41,67,0,7.49,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                236
City                   236
Country                234
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Weather Description    236
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()

City_ID                234
City                   234
Country                234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Weather Description    234
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
12,Georgetown,MY,82.33,moderate rain,5.4112,100.3354,
14,Rikitea,PF,76.30,overcast clouds,-23.1203,-134.9692,
15,Atuona,PF,78.53,few clouds,-9.8000,-139.0333,
17,Sola,VU,80.53,overcast clouds,-13.8833,167.5500,
18,Maceio,BR,83.64,broken clouds,-9.6658,-35.7353,
19,Catuday,PH,82.74,overcast clouds,16.2923,119.8062,
20,Mabaruma,GY,75.22,overcast clouds,8.2000,-59.7833,
21,Madimba,TZ,81.18,broken clouds,-10.4000,40.3333,
22,Acapulco,MX,82.22,broken clouds,16.8634,-99.8901,
23,Karratha,AU,77.41,clear sky,-20.7377,116.8463,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
12,Georgetown,MY,82.33,moderate rain,5.4112,100.3354,Cititel Penang
14,Rikitea,PF,76.30,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Atuona,PF,78.53,few clouds,-9.8000,-139.0333,Villa Enata
17,Sola,VU,80.53,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
18,Maceio,BR,83.64,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió
19,Catuday,PH,82.74,overcast clouds,16.2923,119.8062,Benjie's Kubo & Cottages
20,Mabaruma,GY,75.22,overcast clouds,8.2000,-59.7833,Monty Brooms Guest House
21,Madimba,TZ,81.18,broken clouds,-10.4000,40.3333,
22,Acapulco,MX,82.22,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
23,Karratha,AU,77.41,clear sky,-20.7377,116.8463,ibis Styles Karratha


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
12,Georgetown,MY,82.33,moderate rain,5.4112,100.3354,Cititel Penang
14,Rikitea,PF,76.30,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Atuona,PF,78.53,few clouds,-9.8000,-139.0333,Villa Enata
17,Sola,VU,80.53,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
18,Maceio,BR,83.64,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió
19,Catuday,PH,82.74,overcast clouds,16.2923,119.8062,Benjie's Kubo & Cottages
20,Mabaruma,GY,75.22,overcast clouds,8.2000,-59.7833,Monty Brooms Guest House
21,Madimba,TZ,81.18,broken clouds,-10.4000,40.3333,NaN
22,Acapulco,MX,82.22,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
23,Karratha,AU,77.41,clear sky,-20.7377,116.8463,ibis Styles Karratha


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<d1>Country Code</dt><dd>{Country}</dd>
<dt>City Name</dt><dd>{City}</dd>
<d1>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
<d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))